Dense net Impl

There are certain parts of the code which call the device: "cpu", if you end up using the ecetesla servers, this line will update the device accordingly (i.e. we dont have to worry about it)

device = "cuda" if torch.cuda.is_available() else "cpu"


In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

from torch.utils.data import DataLoader, random_split

sys.path.append("..")
from image_dataset import MultiLabelImageDataset


Hyperparameter init

In [14]:
# Hyperparameters
BATCH_SIZE = 32
LR = 1e-3
NUM_EPOCHS = 3

# File Paths
DATASET_PATH = "../dataset.pt"
MODEL_SAVE_PATH = "densenet_model.pth"
CSV_LOG_PATH = "densenet_training_log.csv"


Using densenet121 adapted for multi-label classification

Use default function params

In [15]:
def create_densenet(num_classes=290, pretrained=True):
    model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT if pretrained else None)

    in_features = model.classifier.in_features
    
    # replace with a linear layer for 290 classes
    model.classifier = nn.Linear(in_features, num_classes)
    return model


In [16]:
def mean_average_precision_at_20(model, loader, device="cpu"):
    model.eval()
    all_ap = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            probs = torch.sigmoid(outputs)

            topk_indices = torch.argsort(probs, dim=1, descending=True)[:, :20]

            for i in range(labels.size(0)):
                true_labels = labels[i].nonzero(as_tuple=True)[0]
                if len(true_labels) == 0:
                    continue

                precision_sum = 0.0
                hits = 0
                for k, idx in enumerate(topk_indices[i]):
                    if idx in true_labels:
                        hits += 1
                        precision_at_k = hits / (k + 1)
                        precision_sum += precision_at_k

                ap = precision_sum / min(len(true_labels), 20)
                all_ap.append(ap)

    return float(np.mean(all_ap)) if len(all_ap) > 0 else 0.0


Training and Eval

** Can comment the line to evaluate map@20 on train if we want to save time 

In [17]:
def train_and_evaluate(
    model,
    train_loader,
    test_loader,
    lr=1e-3,
    num_epochs=3,
    model_save_path=MODEL_SAVE_PATH,
    csv_log_path=CSV_LOG_PATH,
    device="cpu"
):
    """
    Train DenseNet for multi-label classification and evaluate with MAP@20.
    Saves the model and logs to CSV, also plots progress.
    """
    # Send model to GPU if available
    model = model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # For plotting/logging
    epoch_list = []
    train_loss_list = []
    train_map_list = []
    test_map_list = []
    
    # CSV columns
    columns = ["epoch", "train_loss", "train_map20", "test_map20"]
    if not os.path.exists(csv_log_path):
        pd.DataFrame(columns=columns).to_csv(csv_log_path, index=False)
    
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(train_loader.dataset)

        # Evaluate MAP@20 on train set (optional, can be time-consuming)
        train_map20 = mean_average_precision_at_20(model, train_loader, device=device)
        
        # Evaluate MAP@20 on test set
        test_map20 = mean_average_precision_at_20(model, test_loader, device=device)
        
        print(f"Epoch {epoch}/{num_epochs}, "
              f"Train Loss: {avg_loss:.4f}, "
              f"Train MAP@20: {train_map20:.4f}, "
              f"Test MAP@20: {test_map20:.4f}")
        
        epoch_list.append(epoch)
        train_loss_list.append(avg_loss)
        train_map_list.append(train_map20)
        test_map_list.append(test_map20)
        
        # Append to CSV
        df_log = pd.DataFrame([[epoch, avg_loss, train_map20, test_map20]], columns=columns)
        df_log.to_csv(csv_log_path, mode='a', header=False, index=False)
    
    # Save the model weights
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    
    # Plot training loss & test MAP@20
    fig, ax1 = plt.subplots(figsize=(8, 5))

    # Plot training loss
    color1 = "tab:blue"
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Train Loss", color=color1)
    ax1.plot(epoch_list, train_loss_list, color=color1, marker="o", label="Train Loss")
    ax1.tick_params(axis="y", labelcolor=color1)

    # Plot test MAP@20
    ax2 = ax1.twinx()
    color2 = "tab:red"
    ax2.set_ylabel("MAP@20", color=color2)
    ax2.plot(epoch_list, test_map_list, color=color2, marker="x", label="Test MAP@20")
    ax2.tick_params(axis="y", labelcolor=color2)

    plt.title("Train Loss vs Test MAP@20 (DenseNet)")
    plt.show()


Main Code
1. Load Dataset
2. Train/Test split
3. Create Dataloaders
4. Create Densenet model
5. Train & Eval
6. Run

In [18]:
def run_experiment(num_epochs=3, lr=1e-3, device="cpu"):
    dataset = torch.load(DATASET_PATH)
    total_samples = len(dataset)
    print(f"Total samples: {total_samples}")
    
    train_size = int(0.8 * total_samples)
    test_size = total_samples - train_size
    torch.manual_seed(42)
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    print(f"Train set: {len(train_dataset)}, Test set: {len(test_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = create_densenet(num_classes=290, pretrained=True)
    print("DenseNet model created.")
    
    train_and_evaluate(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        lr=lr,
        num_epochs=num_epochs,
        model_save_path=MODEL_SAVE_PATH,
        csv_log_path=CSV_LOG_PATH,
        device=device
    )

device = "cuda" if torch.cuda.is_available() else "cpu"
run_experiment(num_epochs=3, lr=1e-3, device=device)


Total samples: 5950
Train set: 4760, Test set: 1190


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/rianarang/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:11<00:00, 2.75MB/s]


DenseNet model created.


FileNotFoundError: [Errno 2] No such file or directory: 'data/train/14ffd1d1-4a94-42ea-858f-3afc1b313bbe.png'